# training t5 for chinese news summery


In [10]:
# data load
import pandas as pd    
data_sample_path = "/data/agent_h/news2016zh/news2016zh_valid.json"
data = pd.read_json(path_or_buf=data_sample_path, lines=True)
data = data.to_dict('records')

76797


In [29]:
# data explore
from pprint import pprint
import random
print(len(data))
idx = int(random.random()*len(data))
print("id: ", idx)
pprint(data[idx])
print("title: ", data[idx]['title'])
print("content: ", data[idx]['content'][:512])

76797
id:  14784
{'content': '近日，世界贸易组织(WTO)对中国第六次贸易政策审议在日内瓦顺利结束。此次审议过程中，中国经贸体制、贸易投资领域取得的新进展等多方面得到积极评价，各成员对中国成为其重要经贸合作伙伴十分重视。对此，专家指出，虽然上半年中国进出口双下降，但中国在全球贸易经济中的地位仍不断上升，尤其是中国外贸新旧动能转换释放出的强劲动力，将推动对外贸易继续回稳、向好，也将为全球贸易增长作出重要贡献。新动能持续积累优势上半年，我国进出口同比下降3.3%，进口、出口分别下降4.7%和2.1%。虽然进出口双下降，但我国外贸新旧动能转换正加快进程，贸易结构不断优化。海关总署数据显示，1-6月，我国一般贸易进出口占进出口总值的56.4%，比去年同期提升1.2个百分点;民企出口增长3.6%，占出口总值的46.6%，占比继续保持首位。“一般贸易占比的持续上升体现出中国自主产品的比重在上升、自主创新能力在增强，我国对外贸易正向高附加值端发展，如高新技术产业等新业态在我国外贸发展中的势头已越来越强劲。而民企出口的快速发展带来了更多活力，外贸中的国内资本和投入品的增加，为我国外贸健康发展及结构优化提供了新动能。”国家发改委对外经济研究所国际合作室主任张建平在接受本报记者采访时说。多边、双边经贸合作不断拓展则为我国外贸提供了更大发展空间。海关总署新闻发言人黄颂平指出，上半年，我国对部分“一带一路”沿线国家出口增长。另外，已有22个国家或地区与我国签署并实施自贸协定，上半年，与上述国家或地区的进出口表现好于同期我国进出口总体降幅。商务部研究院国际市场研究部副主任白明表示，上半年，大型成套产品出口保持正增长，这个领域的商品技术含量高，附加值也比较高，跟一般的传统商品相比，它更是我们发展的一个方向。跨境电商贸易、市场采购贸易等新型外贸商业模式正成为新的外贸增长点。中国贸促会副会长尹宗华指出，去年我国的跨境电子商务规模为5.4万亿元人民币，预计今年可能会达到6.5万亿元人民币，对于促进外贸稳增长、调结构发挥了重要作用。“机器换人”降低成本随着新旧动能转换的持续推进，传统动能这一曾经的外贸主要贡献者正面临困境。海关总署数据显示，截至今年6月份，我国的加工贸易进口、出口已经分别连续18个月和16个月下降。今年上半年，加工贸易进出口下降9.8%，拖累我国外贸进出口

In [3]:
# end exploring, from now on this is in acutal training script
# create dataset
# TODO test sharded
import pandas as pd    
import transformers
from datasets import load_dataset, load_metric, Dataset
raw_data_train_path = "/data/agent_h/news2016zh/news2016zh_train.json"
raw_data_valid_path = "/data/agent_h/news2016zh/news2016zh_valid.json"

USE_SAMPLE_DATASET = False # use small set 
if USE_SAMPLE_DATASET:
    train_data = pd.read_json(path_or_buf=raw_data_valid_path, lines=True)
    train_data = train_data.to_dict('records')
    train_data = train_data[:]
else:
    train_data = pd.read_json(path_or_buf=raw_data_train_path, lines=True)
    train_data = train_data.to_dict('records')

print("train size: ", len(train_data))

train_data = Dataset.from_list(train_data)
train_data = train_data.train_test_split(test_size=min(5000,int(len(train_data)*0.1)))


train size:  2430752


In [4]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

def preprocess_cn_news_text_data(dataset,
                                 tokenizer_base = "/data/agent_h/llms/umt5-small",
                                 data_key = 'content',
                                 label_key = 'title',
                                 prefix="生成标题：",
                                 max_input_len=512,
                                 max_target_len=64,
                                ):
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_base)
    inputs = [prefix + text for text in dataset[data_key]]
    inputs = tokenizer(inputs,
                       max_length=max_input_len,
                       truncation=True)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(dataset[label_key],
                           max_length=max_input_len, 
                           truncation=True)
    
    inputs["labels"] = labels["input_ids"]
    return inputs

# TODO how to save this thing

model_base = "/data/agent_h/llms/umt5-small"
dataset_path = "/data/agent_h/umt5-small-news2016zh-tokens-full"
# tokenize
train_data = train_data.map(preprocess_cn_news_text_data,batched=True,batch_size=100000)
train_data.save_to_disk(dataset_path)


Map:   0%|                                                                     | 0/2425752 [00:00<?, ? examples/s]/home/agent_h/miniconda3/envs/vsr3/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:3946: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Saving the dataset (1/1 shards): 100%|█████████████████████████████| 5000/5000 [00:00<00:00, 246078.17 examples/s]


In [13]:
# training in 11_t5_exp2.py
# test
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model_ckpt = "/data/agent_h/checkpoints/umt5-small-medium-title-generation/checkpoint-32000"
model_ckpt = "/data/agent_h/vsr2/checkpoints/umt5-small-medium-title-generation-zh/checkpoint-72000"
model = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt)
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

text = """summarize: Combining both modern and traditional style architectures, with one side of the city being modernized and renovated to fit the times, and the other half still offering traditional hutong districts.[18] Beijing is one of the oldest cities in the world, with a rich history dating back over three millennia. As the last of the Four Great Ancient Capitals of China, Beijing has been the political center of the country for most of the past eight centuries,[19] and was the largest city in the world by population for much of the second millennium CE.[20] With mountains surrounding the inland city on three sides, in addition to the old inner and outer city walls, Beijing was strategically poised and developed to be the residence of the emperor and thus was the perfect location for the imperial capital. The city is renowned for its opulent palaces, temples, parks, gardens, tombs, walls and gates.[21] Beijing is one of the most important tourist destinations of the world. In 2018, Beijing was the second highest earning tourist city in the world after Shanghai.[22] Beijing is home to many national monuments and museums and has seven UNESCO World Heritage Sites—the Forbidden City, Temple of Heaven, Summer Palace, Ming Tombs, Zhoukoudian Peking Man Site, and parts of the Great Wall and the Grand Canal—all of which are popular tourist locations.[23] Siheyuans, the city's traditional housing style, and hutongs, the narrow alleys between siheyuans, are major tourist attractions and are common in urban Beijing."""
text = """生成标题：近日，世界贸易组织(WTO)对中国第六次贸易政策审议在日内瓦顺利结束。此次审议过程中，中国经贸体制、贸易投资领域取得的新进展等多方面得到积极评价，各成员对中国成为其重要经贸合作伙伴十分重视。对此，专家指出，虽然上半年中国进出口双下降，但中国在全球贸易经济中的地位仍不断上升，尤其是中国外贸新旧动能转换释放出的强劲动力，将推动对外贸易继续回稳、向好，也将为全球贸易增长作出重要贡献。新动能持续积累优势上半年，我国进出口同比下降3.3%，进口、出口分别下降4.7%和2.1%。虽然进出口双下降，但我国外贸新旧动能转换正加快进程，贸易结构不断优化。海关总署数据显示，1-6月，我国一般贸易进出口占进出口总值的56.4%，比去年同期提升1.2个百分点;民企出口增长3.6%，占出口总值的46.6%，占比继续保持首位。“一般贸易占比的持续上升体现出中国自主产品的比重在上升、自主创新能力在增强，我国对外贸易正向高附加值端发展，如高新技术产业等新业态在我国外贸发展中的势头已越来越强劲。而民企出口的快速发展带来了更多活力，外贸中的国内资本和投入品的增加，为我国外贸健康发展及结构优化提供了新动能。”国家发改委对外经济研究所国际合作室主任张建平在接受本报记者采访时说。多边、双边经贸合作不断拓展则为我国外贸提供了更大发展空间。海关总署新闻发言人黄颂平指出，上半年，我国对部分“一带一路”沿线国家出口增长。另外，已有22个国家或地区与我国签署并实施自贸协定，上半年，与上述国家或地区的进出口表现好于同期我国进出口总体降幅。商务部研究院国际市场研究部副主任白明表示，上半年，大型成套产品出口保持正增长，这个领域的商品技术含量高，附加值也比较高，跟一般的传统商品相比，它更是我们发展的一个方向。跨境电商贸易、市场采购贸易等新型外贸商业模式正成为新的外贸增长点。中国贸促会副会长尹宗华指出，去年我国的跨境电子商务规模为5.4万亿元人民币，预计今年可能会达到6.5万亿元人民币，对于促进外贸稳增长、调结构发挥了重要作用。“机器换人”降低成本随着新旧动能转换的持续推进，传统动能这一曾经的外贸主要贡献者正面临困境。海关总署数据显示，截至今年6月份，我国的加工贸易进口、出口已经分别连续18个月和16个月下降。今年上半年，加工贸易进出口下降9.8%，拖累我国外贸进出口整体下降约3个百分点。“加工贸易等传统动能对于当前的中国外贸而言仍很重要，它既可以推动贸易均衡发展，也能够为就业提供保障。对于加工贸易的下降，我们在顺应市场规律的前提下，还要充分发掘其潜力，这包括了结构的改善与量的增长。”张建平说，而要充分发掘这一潜力，则要在保留优势的基础上提高其在价值链中的地位，并不断提高贸易便利化水平。 """
text = """生成标题：据深圳证券交易所近日公告，安徽晶奇网络科技股份有限公司在中国证监会审阅其IPO并在创业板上市申请文件的过程中，该公司与其保荐机构主动要求撤回注册申请文件。值得注意的是，该企业早在2021年就已“过会”。此外，浙江控阀2022年12月创业板IPO过会，一年多未提交注册，今年3月撤回IPO；博菱电器2022年11月创业板IPO过会，过会逾一年未提交注册，最终今年3月撤回IPO。中国人民大学中国资本市场研究院联席院长赵锡军在接受中新社直通车记者采访时表示，从公开资料来看，上述企业之所以主动撤回IPO申请，主要是在IPO自查过程中，发现公司在合规、板块定位、信息披露、会计处理等方面存在问题，及时纠正。企业之所以在IPO问题上如此积极自查自纠，这与当前中国资本市场“严监管”的风气密切相关。"""
# text = """最近，不少凯迪拉克经销商在社交媒体上宣称，目前正在开展限量促销活动，该品牌产品线中价格最亲民的车型——CT4，落地价已经低于16万元了。要知道，作为美式运动轿车的CT4，目前厂商指导价21.97万～25.97万元。这么一看，相当于直降了约8万元。由于上述消息出自某4S店的官方账号，瞬间吸引了不少潜在消费者的关注，但在社交媒体上，网友没有看到太多车主分享相关内容，于是纷纷在评论区留言，求证真实性。带着疑问，近日「市界」走访了北京多家凯迪拉克4S店，销售董磊表示，虽然实际的成交价格和不同地区、不同4S店经销商的自身情况有关，但现在CT4正在促销的确是真的。“CT4这个车卖的不好，目前厂家处于停产、清库存的状态，是否还会继续生产并没有确定的消息，所以优惠力度比较大。”据董磊分析，13万多的裸车价，大概率是需要以分期贷款的方式购车，“落地价在15万多的话，估计是没把利息算进去。” """
# text = """生成标题：热。酒红色的光晕压得我透不过气，那些山也东倒西歪地躺在一边。大的小的石头都一动不动，只有扭曲而上的草臭，使人无止境地口渴。我拖着一背包的空瓶子独自行走，独自行走于天地之间。我有一种全身赤裸的冲动，仿佛连文明的痕迹都被这里排斥。我践踏这个醉醺醺的世界，而避开那些动植物的尸骸。恐怕行程尚远，我想，而自然原来是如此残酷。那些拼了命在呼吸的尘土，甚至能灼伤我的眼睛。它们向四面八方扩张，猎杀了我视线之内的一切凉意。而当地表开始倾斜，黄昏也同瘟疫一般弥漫。我不惧怕入夜，因为尚有火焰在我胸中燃烧。"""
# text = """生成标题：穿过森林后，我看到了海。是在远处不断升起的海，甚至成为了月亮的背景。如同发酵过般的乳黄色的月光洒向海面。有一些纤细的鸟类啄食着，它们的羽毛也是月光的颜色。风温柔而慵懒，吹动了树木的影子。星星稀稀散散，连着那一小点碎云彩，一同被水声冲刷到了车窗上。浪迎面而来，又后退而去，而后又淹没在列车的行进中了。没有人与我分享这份景色。夜已经很深了，一切都变得缓慢。除了时间。她总是不曾等待我。"""
inputs = tokenizer(
    text,
    return_tensors="pt",
)
outputs = model.generate(**inputs)
print(text)
print("output: ")
print(tokenizer.batch_decode(outputs))

生成标题：据深圳证券交易所近日公告，安徽晶奇网络科技股份有限公司在中国证监会审阅其IPO并在创业板上市申请文件的过程中，该公司与其保荐机构主动要求撤回注册申请文件。值得注意的是，该企业早在2021年就已“过会”。此外，浙江控阀2022年12月创业板IPO过会，一年多未提交注册，今年3月撤回IPO；博菱电器2022年11月创业板IPO过会，过会逾一年未提交注册，最终今年3月撤回IPO。中国人民大学中国资本市场研究院联席院长赵锡军在接受中新社直通车记者采访时表示，从公开资料来看，上述企业之所以主动撤回IPO申请，主要是在IPO自查过程中，发现公司在合规、板块定位、信息披露、会计处理等方面存在问题，及时纠正。企业之所以在IPO问题上如此积极自查自纠，这与当前中国资本市场“严监管”的风气密切相关。
output: 
['<pad> 晶奇网络科技公司主动撤回IPO申请</s>']
